In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import RMSprop
from sklearn.model_selection import StratifiedKFold

In [2]:
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)

In [3]:
podcasts_segments = np.load('mfcc_x.npy')
ads_segments = np.load('mfcc_y.npy')

In [4]:
#X_train, X_test, y_train, y_test = train_test_split(np.array(podcasts_segments), np.array(ads_segments), test_size=0.10, random_state=42)


#X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)
#print(X_train.shape)
#print(X_val.shape)
#print(X_test.shape)
#X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2],1)
#X_val = X_val.reshape(X_val.shape[0], X_val.shape[1], X_val.shape[2],1)
#X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2],1)
#print(X_train.shape)

In [5]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 10, 1)))
#model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
#model.add(layers.Conv2D(64, (3, 3), activation='relu'))


model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1))

model.summary()

def get_model():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(20, 10, 1)))
    #model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    #model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))


    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))

    model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate=0.0001),
              metrics='accuracy')

    return model

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 8, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 124, 6, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 62, 3, 64)        0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 11904)             0         
                                                                 
 dense (Dense)               (None, 64)                761920    
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                        

In [6]:
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate=0.0001),
              metrics='accuracy')
              
#history = model.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val))


In [7]:
def get_evaluation(model, X_train, X_test, y_train, y_test):
    history = model.fit(X_train,y_train,epochs=20)
    return model.evaluate(X_test, y_test)

In [8]:
history_arr = []

for train_index, test_index in kfold.split(np.array(podcasts_segments), np.array(ads_segments)):
    X_train, X_test, y_train, y_test = np.array(podcasts_segments)[train_index], np.array(podcasts_segments)[test_index], \
                                        np.array(ads_segments)[train_index], np.array(ads_segments)[test_index]
    
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2],1)
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2],1)
    
    history_arr.append(get_evaluation(get_model(),X_train,X_test,y_train,y_test))

Epoch 1/20


ValueError: in user code:

    File "C:\Users\morte\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\morte\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\morte\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\morte\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training.py", line 859, in train_s

In [ ]:
history_arr

[[1.4396694898605347, 0.7073761820793152],
 [3.687818765640259, 0.666079044342041],
 [1.7904168367385864, 0.74411940574646],
 [1.281252145767212, 0.813948392868042]]

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.3, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

NameError: name 'history' is not defined